In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [23]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [15]:
STARTINGCSV=pd.read_csv('DailyPlayerStatsAsOf29Dec2023.csv')  
STARTINGCSV

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,shorthandedGoals,shorthandedPoints,opponentAbbrev,pim,toi,playerId,firstName,lastName
0,2023020529,NYR,H,2023-12-27,0,1,{'default': 'Rangers'},{'default': 'Capitals'},1,1,0,0,0,0,2,16,0,0,WSH,0,11:08,8471218,Blake,Wheeler
1,2023020522,NYR,H,2023-12-23,0,1,{'default': 'Rangers'},{'default': 'Sabres'},1,0,0,0,0,0,0,20,0,0,BUF,0,15:57,8471218,Blake,Wheeler
2,2023020511,NYR,H,2023-12-22,1,0,{'default': 'Rangers'},{'default': 'Oilers'},1,1,0,0,0,0,3,17,0,0,EDM,0,12:04,8471218,Blake,Wheeler
3,2023020489,NYR,R,2023-12-19,0,2,{'default': 'Rangers'},{'default': 'Maple Leafs'},2,2,0,0,0,0,1,19,0,0,TOR,0,11:34,8471218,Blake,Wheeler
4,2023020462,NYR,R,2023-12-16,0,0,{'default': 'Rangers'},{'default': 'Bruins'},0,0,0,0,0,0,1,16,0,0,BOS,0,11:55,8471218,Blake,Wheeler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19052,2023020074,SJS,R,2023-10-21,0,0,{'default': 'Sharks'},{'default': 'Predators'},0,-1,0,0,0,0,1,19,0,0,NSH,0,13:55,8482667,William,Eklund
19053,2023020062,SJS,H,2023-10-19,0,0,{'default': 'Sharks'},{'default': 'Bruins'},0,-1,0,0,0,0,0,22,0,0,BOS,0,20:01,8482667,William,Eklund
19054,2023020047,SJS,H,2023-10-17,1,0,{'default': 'Sharks'},{'default': 'Hurricanes'},1,-1,1,1,0,0,2,19,0,0,CAR,0,16:00,8482667,William,Eklund
19055,2023020030,SJS,H,2023-10-14,0,0,{'default': 'Sharks'},{'default': 'Avalanche'},0,1,0,0,0,0,1,18,0,0,COL,0,14:14,8482667,William,Eklund


In [24]:
playerStats=STARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','shots','opponentAbbrev']]

playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('teamAbbrev').GameNumber.transform(np.max)

playerStats['SeasonAvgSOG']= playerStats.groupby(['playerId']).shots.transform(np.sum)/playerStats['GP']
playerStats['OverallZScoreShots'] = np.abs(stats.zscore(playerStats['shots']))

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)

playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).shots.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots
0,NYR,8471218,Blake,Wheeler,2023-12-27,2,WSH,33,33,1.575758,0.174938,1,1.6
1,NYR,8471218,Blake,Wheeler,2023-12-23,0,BUF,32,33,1.575758,1.105072,1,1.6
2,NYR,8471218,Blake,Wheeler,2023-12-22,3,EDM,31,33,1.575758,0.814942,1,1.6
3,NYR,8471218,Blake,Wheeler,2023-12-19,1,TOR,30,33,1.575758,0.465067,1,1.6
4,NYR,8471218,Blake,Wheeler,2023-12-16,1,BOS,29,33,1.575758,0.465067,1,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19052,SJS,8482667,William,Eklund,2023-10-21,1,NSH,5,36,1.638889,0.465067,0,2.6
19053,SJS,8482667,William,Eklund,2023-10-19,0,BOS,4,36,1.638889,1.105072,0,2.6
19054,SJS,8482667,William,Eklund,2023-10-17,2,CAR,3,36,1.638889,0.174938,0,2.6
19055,SJS,8482667,William,Eklund,2023-10-14,1,COL,2,36,1.638889,0.465067,0,2.6


In [29]:
res = playerStats.groupby(['playerId'])["shots"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['shots'])&(res['shots']<res['High'])].drop(columns=['Low', 'High'])

ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['shots'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).shots.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,NYR,8471218,Blake,Wheeler,2023-12-27,2,WSH,33,33,1.575758,0.174938,1,1.6,22,22,1.590909
2,NYR,8471218,Blake,Wheeler,2023-12-22,3,EDM,31,33,1.575758,0.814942,1,1.6,21,22,1.590909
3,NYR,8471218,Blake,Wheeler,2023-12-19,1,TOR,30,33,1.575758,0.465067,1,1.6,20,22,1.590909
4,NYR,8471218,Blake,Wheeler,2023-12-16,1,BOS,29,33,1.575758,0.465067,1,1.6,19,22,1.590909
5,NYR,8471218,Blake,Wheeler,2023-12-15,1,ANA,28,33,1.575758,0.465067,1,1.6,18,22,1.590909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19051,SJS,8482667,William,Eklund,2023-10-24,1,FLA,6,36,1.638889,0.465067,0,2.6,5,27,1.777778
19052,SJS,8482667,William,Eklund,2023-10-21,1,NSH,5,36,1.638889,0.465067,0,2.6,4,27,1.777778
19054,SJS,8482667,William,Eklund,2023-10-17,2,CAR,3,36,1.638889,0.174938,0,2.6,3,27,1.777778
19055,SJS,8482667,William,Eklund,2023-10-14,1,COL,2,36,1.638889,0.465067,0,2.6,2,27,1.777778


In [34]:
shotSummary=ShotsNoOutliers[['teamAbbrev',	'playerId',	'firstName', 'lastName', 'SeasonAvgSOG', 'LastFiveAvgShots', 'NoOutliersAvgSOG']].drop_duplicates()
shotSummary.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,NoOutliersAvgSOG
0,BOS,8477956,David,Pastrnak,5.030303,6.0,5.230769
1,TBL,8476453,Nikita,Kucherov,4.500000,4.4,5.037037
2,NJD,8481559,Jack,Hughes,3.878788,1.6,4.818182
3,COL,8477492,Nathan,MacKinnon,4.485714,5.6,4.592593
4,OTT,8480801,Brady,Tkachuk,4.633333,5.8,4.375000
5,TOR,8479318,Auston,Matthews,4.187500,4.6,4.269231
6,TOR,8477939,William,Nylander,4.312500,5.6,4.238095
7,NYR,8478550,Artemi,Panarin,4.090909,6.0,4.185185
8,VGK,8478403,Jack,Eichel,4.135135,4.4,3.933333
9,WPG,8478398,Kyle,Connor,3.090909,0.0,3.900000
